Question 1

For the second half of FY23 (Apr - Sep), what is our gross charge split for peds (\<18 yrs old on date of admit) vs other patients?

Notes:

After verifying the BIRTH\_DT column for NULL values and finding none, an INNER JOIN was chosen instead of a LEFT JOIN for the join operation.

To distinguish between pediatric and adult patients, CASE statements were employed. If the patient's age is 18 years or older, they are categorized as adults.

Since the question concerns FY (fiscal year) dates, the PostingDate column was utilized to filter the data appropriately.

In [27]:
SELECT
    CASE
        WHEN DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) >= 18 THEN 'Other Patients'
        ELSE 'Peds (<18 yrs old)'
    END AS PatientType,
    SUM(T.LineCharges) AS TotalCharges
FROM
    dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN 
        ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    INNER JOIN dbo.Patient AS P 
        ON AN.MRN = P.MRN
WHERE
    T.PostingDate >= '2023-04-01'
    AND T.PostingDate < '2023-10-01'
GROUP BY
    CASE
        WHEN DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) >= 18 THEN 'Other Patients'
        ELSE 'Peds (<18 yrs old)'
    END;

(2 rows affected)

Total execution time: 00:00:00.066

PatientType,TotalCharges
Peds (<18 yrs old),7398025.83
Other Patients,22954497.97


Question 2

What revenue code incurred the most gross charges for patients over 65 years old and insured by a Medicare product in FY23 (Oct - Sep)?

Notes:

In this query, the assumption is made that 'Medicare' in the AN.PRIM\_FIN\_CLASS\_NM column represents the correct financial class grouping for Medicare products.

There is a potential candidate to include in the query, which is 'Medicare Managed' or Advantage Plans. However, further clarification would be required to determine its inclusion.

When both 'Medicare' and 'Medicare Managed' are included in the query, the revenue code with the highest gross charges is 0301 - Laboratory - Clinical Chemistry, with a total of 426,462.15 in charges.

Alternatively, when considering only 'Medicare' without 'Medicare Managed', the highest gross charges are associated with Revcode 202 - Intensive Care Unit, totaling 354,744.00 in charges.

In [26]:
SELECT TOP (1)
	T.Revcode,
	RC.REVENUE_CODE_NAME AS RevenueCodeName,
	SUM(T.LineCharges) AS TotalCharges
FROM 
	dbo.Transactions AS T
	INNER JOIN dbo.Account_Number AS AN
		ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
	INNER JOIN dbo.Patient AS P
		ON AN.MRN = P.MRN
	INNER JOIN dbo.Rev_Codes AS RC
		ON T.Revcode = RC.REVENUE_CODE
WHERE 
	P.BIRTH_DT <= DATEADD(year, -65, GETDATE())
	AND AN.PRIM_FIN_CLASS_NM = ('Medicare')
	AND T.PostingDate >= '2022-10-01'
	AND T.PostingDate < '2023-10-01'
GROUP BY 
	T.Revcode,
	RC.REVENUE_CODE_NAME
ORDER BY 
	TotalCharges DESC;


(1 row affected)

Total execution time: 00:00:00.167

Revcode,RevenueCodeName,TotalCharges
0202,INTENSIVE CARE-MEDICAL,354744.00


Question 3

Which provider generated the most gross charges for female patients in the first half of FY23 (Oct - Mar)?

Notes: 

To retrieve the Provider Description column, three joins are required.

When joining the Account\_Number table with the Provider table using VST\_PROV\_NPI, approximately 5,722 rows with blank VST\_PROV\_NPI values will be discarded. This is not a problem as the focus is on the top provider with the highest charges.

In [16]:
SELECT TOP (1)
    P.[Provider Description] AS ProviderDescription,
    SUM(T.LineCharges) AS TotalCharges
FROM
    dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN 
		  ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    INNER JOIN dbo.Patient AS PT 
		  ON AN.MRN = PT.MRN
    INNER JOIN dbo.Provider AS P 
		  ON AN.VST_PROV_NPI = P.NPI
WHERE
	T.PostingDate >= '2022-10-01'
    AND T.PostingDate < '2023-04-01'
    AND PT.SEX = 'F'
GROUP BY
    P.[Provider Description]
ORDER BY
	TotalCharges DESC;

(1 row affected)

Total execution time: 00:00:00.130

ProviderDescription,TotalCharges
PROV_0951,187397.97


Question 4

How many unique patients were admitted to the hospital and incurred a room and board charge (defined via rev code) in FY23?

Notes:

To count the total number of unique patients based on admission, the Account\_Number table is used. Joining with the Transactions table and then the Rev\_Codes table allows retrieval of the REVENUE\_CODE\_NAME for filtering on the Room & Board revenue code.

To ensure each patient is counted only once, the DISTINCT keyword is applied to the MRN column.

Different approaches can be taken to filter on the room and board charge. One option is using the LIKE keyword on the \[Revcode Desc\] in the transactions table. Alternatively, the IN statement can be used after identifying the appropriate revenue codes. In this case, the LIKE keyword on the REVENUE\_CODE\_NAME is utilized to demonstrate the table relationships.

In [20]:
SELECT 
	COUNT(DISTINCT AN.MRN) AS UniquePatients
FROM dbo.Account_Number AS AN
	INNER JOIN dbo.Transactions AS T 
		ON AN.ACCOUNT_NUMBER = T.ACCOUNT_NUMBER
	INNER JOIN dbo.Rev_Codes AS RC 
		ON T.Revcode = RC.REVENUE_CODE
WHERE T.PostingDate >= '2022-10-01'
    AND T.PostingDate < '2023-10-01'
    AND RC.REVENUE_CODE_NAME LIKE 'ROOM & BOARD%';

(1 row affected)

Total execution time: 00:00:00.074

UniquePatients
375


Question 5

What month had the most inpatient account volume where total charges per account were \>$25,000?

Notes: 

In this query, there are a few important assumptions made:

1. The ACCT\_DICSH\_DT column is assumed to represent the month since the question does not explicitly mention which date field to use. It's important to note that using a different date field may yield different results.
    
2. The TOT\_CHGS column is chosen as the basis for the WHERE statement. The focus of the question is on inpatient account volumes, not specific details such as transactions, posting dates, CPT codes, or revenue codes. Using the TOT\_CHGS column as a filter is considered safe because it aligns with the total in the Transactions table.
    
3. The VISIT\_BASECLS\_NM column is assumed to be the most suitable for filtering inpatient accounts. This choice is based on the limited number of options in the column, which makes grouping and analysis more straightforward.

In [14]:
SELECT TOP (1)
	DATENAME(MONTH, ACCT_DISCH_DT) AS DischargeMonth,
	COUNT(DISTINCT ACCOUNT_NUMBER) AS AccountVolume
FROM dbo.Account_Number AS AN
WHERE 
	TOT_CHGS > 25000
	AND VISIT_BASECLS_NM = 'Inpatient'
GROUP BY
	DATENAME(MONTH, ACCT_DISCH_DT)
ORDER BY AccountVolume DESC;
	

(1 row affected)

Total execution time: 00:00:00.028

DischargeMonth,AccountVolume
March,33


Bonus 1 

Create a script that will be accessed by Tableau that includes the following fields: 

|  |
| --- |
| Account Number |
| Rev Code |
| Rev Code Description |
| Age at Admission |
| Adm Date |
| Disch Date |
| CPT Code |
| Visit Provider |
| Visit Provider Description |
| Total Charges |

Notes:

The best option is to create a view to allow Tableau to connect to said view. Views offer a variety of benefits such as data consistency, security, performance, and ease of use.

Utilized a LEFT OUTER JOIN on Provider table to ensure that all records from the Transactions and Account\_Number tables are included in the view.

In [10]:
CREATE VIEW dbo.vTableau
AS
SELECT
    AN.ACCOUNT_NUMBER AS [Account Number],
    T.Revcode AS [Rev Code],
    T.[RevCode Desc] AS [Revenue Code Description],
	DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) AS [Age at Admission],
    AN.ACCT_ADMIT_DT AS [Adm Date],
    AN.ACCT_DISCH_DT AS [Disch Date],
    T.CPT AS [CPT Code],
    PN.[Provider Description] AS [Visit Provider],
    PN.[EPIC_PROV_TYPE_NM] AS [Visit Provider Description],
    SUM(T.LineCharges) AS [Total Charges]
FROM dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN
        ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    LEFT OUTER JOIN dbo.Provider AS PN
        ON AN.VST_PROV_NPI = PN.NPI
    INNER JOIN dbo.Patient AS P
        ON AN.MRN = P.MRN
GROUP BY 
    AN.ACCOUNT_NUMBER,
    T.Revcode,
    T.[RevCode Desc],
	P.BIRTH_DT,
    AN.ACCT_ADMIT_DT,
    AN.ACCT_DISCH_DT,
    T.CPT,
    PN.[Provider Description],
    PN.[EPIC_PROV_TYPE_NM];

Commands completed successfully.

Total execution time: 00:00:00

Bonus 2

Create a database design map of all these data elements and how you think they would look in the database

This is included as a jpeg in the github repository labeled: SQL\_DEVELOPER\_SAMPLE.jpeg